In [39]:
#manipulate dataframes in python
import pandas as pd
import sys

#make API calls with python
import requests

# sys.path.insert(0, '/path/to/application/app/folder') used to import local python file, in this case local keys
sys.path.insert(0, '../src/')
import localvars

#allows us to store results of API call cleanly
import json,csv

# Load the first sheet of the JSON file into a data frame
df = pd.read_json('../src/data/data.json', orient='index')

# List of Years
strYears =["2010","2013", "2014", "2015", "2016", "2017"]

intState = 47
intCounty = 125

for index, row in df.iterrows():
    tablename= row['table']
    # Using for loop 
    metric = pd.DataFrame() #creates a new dataframe that's empty
    for i in strYears: 

        #construct the API call we will use
        if tablename[:1]=="B":
            baseAPI = "https://api.census.gov/data/%s/acs/acs5?get=%s&for=tract:*&in=state:%i%%20county:%i&key=%s" 
        elif tablename[:1]=="D":
            baseAPI = "https://api.census.gov/data/%s/acs/acs5/profile?get=%s&for=tract:*&in=state:%i%%20county:%i&key=%s" 

        calledAPI = (baseAPI % (i, tablename, intState, intCounty, localvars.apiKey))

        #call the API and collect the response
        response = requests.get(calledAPI)

        #load the response into a JSON, ignoring the first element which is just field labels
        try:
            formattedResponse = json.loads(response.text)[1:]

            #flip the order of the response from [population, zipcode] -> [zipcode, population]
            formattedResponse = [item[::-1] for item in formattedResponse]

            #store the response in a dataframe
            strColYear = "y_" + i

            metrictemp = pd.DataFrame(columns=['id', '2', '3', strColYear], data=formattedResponse)
            del metrictemp['2']
            del metrictemp['3']
            if i == "2010":
                metric=metrictemp
            else: 
                metric = metric.merge(metrictemp, on='id', how='outer')
        except:
            print("Unexpected error:", sys.exc_info()[0])
            raise

    #save that dataframe to a CSV spreadsheet
    strFileName=str(row['metric'])
    strFile = "../src/data/ProcessedData/r" + strFileName + ".csv"
    strJson ="../src/data/json/m" + strFileName + ".json"
    metric.to_csv(strFile, index=False)
    
    headers = ['id', 'y_2010', 'y_2013', 'y_2014', 'y_2015', 'y_2016', 'y_2017'] 
    dtypes = {'id':str}
    
    df = pd.read_csv(strFile, index_col='id', names=headers, dtype=dtypes)

    df["y_2010"] = pd.to_numeric(df['y_2010'], errors='coerce').fillna(0)
    df["y_2013"] = pd.to_numeric(df['y_2013'], errors='coerce').fillna(0)
    df["y_2014"] = pd.to_numeric(df['y_2014'], errors='coerce').fillna(0)
    df["y_2015"] = pd.to_numeric(df['y_2015'], errors='coerce').fillna(0)
    df["y_2016"] = pd.to_numeric(df['y_2016'], errors='coerce').fillna(0)
    df["y_2017"] = pd.to_numeric(df['y_2017'], errors='coerce').fillna(0)

    export = df.to_json(strJson, orient='index')
    print("Output: " + strJson)

    
    

Output: ../src/data/json/m1.json
Output: ../src/data/json/m2.json
Output: ../src/data/json/m3.json
